# Spark DataFrame 

Spark DataFrames são a principal forma de trabalhar com Spark e Python pós Spark 2.0. DataFrames atuam como versões poderosas de tabelas, com linhas e colunas, lidando facilmente com grandes conjuntos de dados. A mudança para DataFrames oferece muitas vantagens:
* Uma sintaxe muito mais simples
* Capacidade de usar SQL diretamente no dataframe
* As operações são distribuídas automaticamente entre RDDs
    
Se você já usou R ou mesmo a biblioteca pandas com Python, provavelmente já está familiarizado com o conceito de DataFrames. O Spark DataFrame expande muitos desses conceitos, permitindo que você transfira esse conhecimento facilmente ao compreender a sintaxe simples dos DataFrames do Spark. Lembre-se de que a principal vantagem de usar Spark DataFrames em comparação com outros programas é que o Spark pode manipular dados em muitos RDDs, enormes conjuntos de dados que nunca caberiam em um único computador. Isso tem um pequeno custo de algumas opções de sintaxe "peculiares", mas após este curso você se sentirá muito confortável com todos esses tópicos!

Vamos começar!

## Criando um DataFrame

Primeiro, precisamos iniciar uma SparkSession:

In [2]:
from pyspark.sql import SparkSession

Em seguida, inicie o SparkSession

In [3]:
# Pode demorar um pouco em um computador local
spark = SparkSession.builder.appName("Basics").getOrCreate()

Primeiro, você precisará obter os dados de um arquivo (ou se conectar a um grande arquivo distribuído como HDFS, falaremos sobre isso mais tarde, uma vez que migrarmos para conjuntos de dados maiores no AWS EC2).

In [4]:
# Discutiremos como ler outras opções mais tarde.
# Este conjunto de dados é de exemplos do Spark

# Pode ser um pouco lento localmente
df = spark.read.json('people.json')

#### Mostrando os dados

In [6]:
# Observe como os dados estão faltando!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.columns

['age', 'name']

In [9]:
df.describe()

DataFrame[summary: string, age: string, name: string]

Alguns tipos de dados facilitam a inferência do esquema (como formatos tabulares, como csv, que mostraremos mais tarde).

No entanto, muitas vezes você mesmo precisa definir o esquema se não estiver lidando com um método .read que não tenha inferSchema () embutido.

O Spark tem todas as ferramentas de que você precisa para isso, requer apenas uma estrutura muito específica:

In [10]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Em seguida, precisamos criar a lista de campos de estrutura
     *: nome do parâmetro: string, nome do campo.
     *: param dataType:: class: `DataType` do campo.
     *: param nullable: boolean, se o campo pode ser null (None) ou não.

In [11]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [12]:
final_struc = StructType(fields=data_schema)

In [13]:
df = spark.read.json('people.json', schema=final_struc)

In [14]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Pegando os dados

In [15]:
df['age']

Column<b'age'>

In [16]:
type(df['age'])

pyspark.sql.column.Column

In [17]:
df.select('age')

DataFrame[age: int]

In [18]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [19]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [20]:
# Retorna uma lista de objetos Row
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

Várias colunas:

In [21]:
df.select(['age','name'])

DataFrame[age: int, name: string]

In [22]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Criando novas colunas

In [23]:
# Adicionando uma nova coluna com uma cópia simples
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [24]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [25]:
# Renomear Simples
df.withColumnRenamed('age','supernewage').show()

+-----------+-------+
|supernewage|   name|
+-----------+-------+
|       null|Michael|
|         30|   Andy|
|         19| Justin|
+-----------+-------+



Operações mais complicadas para criar novas colunas

In [26]:
df.withColumn('doubleage',df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [27]:
df.withColumn('add_one_age',df['age']+1).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
+----+-------+-----------+



In [28]:
df.withColumn('half_age',df['age']/2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+



In [29]:
df.withColumn('half_age',df['age']/2)

DataFrame[age: int, name: string, half_age: double]

Discutiremos operações muito mais complicadas posteriormente!

### Usando SQL

Para usar consultas SQL diretamente com o dataframe, você precisará registrá-lo em uma visualização temporária:

In [31]:
# Registre o DataFrame como uma visão SQL temporária

df.createOrReplaceTempView("people")

In [32]:
sql_results = spark.sql("SELECT * FROM people")

In [33]:
sql_results

DataFrame[age: int, name: string]

In [34]:
sql_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [35]:
spark.sql("SELECT * FROM people WHERE age=30").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



Não vamos nos concentrar no uso da sintaxe SQL para este curso em geral, mas tenha em mente que ela está sempre lá para você se livrar rapidamente de suas habilidades em SQL!